## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.7435,-97.8558,42.96,75,75,13.80,broken clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,44.26,81,75,23.02,light rain
2,2,Vaini,TO,-21.2000,-175.2000,77.16,94,75,8.05,light rain
3,3,Grindavik,IS,63.8424,-22.4338,36.07,75,90,13.80,light rain
4,4,Georgetown,MY,5.4112,100.3354,78.73,96,40,2.30,thunderstorm with rain


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df= city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & 
                                    (city_data_df["Max Temp"]>= min_temp)]
preferred_city_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vaini,TO,-21.2000,-175.2000,77.16,94,75,8.05,light rain
4,4,Georgetown,MY,5.4112,100.3354,78.73,96,40,2.30,thunderstorm with rain
6,6,Hilo,US,19.7297,-155.0900,75.40,91,75,6.91,light rain
7,7,Cayenne,GF,4.9333,-52.3333,87.84,81,75,5.99,broken clouds
16,16,Sarangani,PH,5.4033,125.4636,83.34,78,31,18.50,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
680,680,Aswan,EG,24.0934,32.9070,85.30,20,0,10.36,clear sky
682,682,Thinadhoo,MV,0.5333,72.9333,83.03,70,17,15.41,few clouds
686,686,Sibolga,ID,1.7427,98.7792,75.45,92,100,1.54,overcast clouds
692,692,Mitu,CO,1.1983,-70.1733,76.24,87,98,3.65,overcast clouds


In [19]:
# 4a. Determine if there are any empty rows.
preferred_city_df.count()

City_ID                227
City                   227
Country                226
Lat                    227
Lng                    227
Max Temp               227
Humidity               227
Cloudiness             227
Wind Speed             227
Current Description    227
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_city_clean_df = preferred_city_df.dropna()
preferred_city_clean_df.count()

City_ID                226
City                   226
Country                226
Lat                    226
Lng                    226
Max Temp               226
Humidity               226
Cloudiness             226
Wind Speed             226
Current Description    226
dtype: int64

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_city_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,77.16,light rain,-21.2000,-175.2000,
4,Georgetown,MY,78.73,thunderstorm with rain,5.4112,100.3354,
6,Hilo,US,75.40,light rain,19.7297,-155.0900,
7,Cayenne,GF,87.84,broken clouds,4.9333,-52.3333,
16,Sarangani,PH,83.34,scattered clouds,5.4033,125.4636,
17,Ranong,TH,78.37,broken clouds,9.9658,98.6348,
19,Kapaa,US,77.32,scattered clouds,22.0752,-159.3190,
22,Kahului,US,78.84,few clouds,20.8947,-156.4700,
31,Trairi,BR,86.32,broken clouds,-3.2778,-39.2689,
39,Katherine,AU,80.92,overcast clouds,-14.4667,132.2667,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [47]:
# 7. Drop the rows where there is no Hotel Name.
# Set empty to cells in in Hotel Name column to NAN
clean_hotel_df = hotel_df.replace('', np.nan)
#Drop the NAN Hotel rows
clean_hotel_df.dropna(subset=["Hotel Name"],inplace = True)
#Count the rows to ensure No NAN rows exist.
clean_hotel_df.count()

City                   207
Country                207
Max Temp               207
Current Description    207
Lat                    207
Lng                    207
Hotel Name             207
dtype: int64

In [49]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
https://maps.googleapis.com/maps/api/directions/json?origin=Disneyland&destination=Universal+Studios+Hollywood&key=